In [1]:
import pandas as pd 
import pickle
import json
import os
import os.path
import numpy as np
import seaborn as sns
from collections import defaultdict
import joblib
import matplotlib.pyplot as plt
from matplotlib.ticker import (
                               FormatStrFormatter, 
                               AutoMinorLocator,
                               FuncFormatter,
                               )
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
%matplotlib inline
from scipy.spatial import distance
import math


import os
os.chdir('G:\My Drive\MSc_project\.MAIN')


c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\berke\anaconda3\envs\env-pytorch\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [14]:
t_df = pd.read_csv(f'multisource_analysis/data/avengers_scored_tweets.csv')
u_df = pd.read_csv(f'multisource_analysis/user_ft_data/avengers_scored_tweets2.csv')

user_columns = [ x for x in u_df.columns.tolist() if x not in t_df.columns.tolist() ]
print(len(user_columns))
user_cols = [ item for item in user_columns if 'mode' not in item ] 
user_cols.remove('user_sentences_mean')
user_cols.remove('user_paragraphs_mean')

84


# USER FEATURES

In [7]:

hashtags = ['avengers','blm','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']


for h_i,hashtag in enumerate(hashtags):

    cats = ['sentiment', 'topic', 'topic_single', 'irony', 'emoji', 'offensive','gender', 'user_topic_mode', 'user_sentiment_mode']

    df = pd.read_csv(f'multisource_analysis/user_ft_data/{hashtag}_scored_tweets2.csv')
    df.drop(['Unnamed: 0', 'text','hashtag','user_id']+cats, inplace = True, axis =1)

    # readability statistics are nan when the tweet contains no text. so change these scores to 0
    reads = ['ARI','LIX','RIX','complex_words','Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']
    for r in reads:
        df[[r]] = df[[r]].fillna(value=0).copy()

    cases = list(set(df['target'].tolist()))

    infectors = df[df['is-target']==1]
    informers = df[df['is-informer']==1]

    n = len(infectors)

    best_formers = []
    cols = infectors.columns.tolist()
    [ cols.remove(x) for x in ['target','is-target','is-infector','is-informer'] ]
    

    for case_id in cases:

        o_fect = infectors.copy()[infectors['target']==case_id]
        # fect = o_fect[user_cols].to_numpy()
        fect = o_fect[cols].to_numpy() 

        o_formers = informers.copy()[informers['target']==case_id]
        # formers = o_formers[user_cols].to_numpy()
        formers = o_formers[cols].to_numpy()


        # get the closest feature to the target per feature
        best_calc = abs(fect - formers)
        best_idx = [ np.argmin(abs(row.astype(float))) for row in best_calc.T ]
        best_former = np.array( [form[best_idx[i]] for i,form in enumerate(formers.T) ] ) # we keeep the informer with the closer scores per feature

        best_formers.append(best_former.tolist())


    best_informers_df = pd.DataFrame(best_formers)
    best_informers_df.columns = cols # replace this with user_cols if just want user features
    best_informers_df.insert(0,column = 'is-target', value= [0]*n )
    best_informers_df.insert(0,column = 'is-infector', value= [0]*n )
    best_informers_df.insert(0,column = 'is-informer', value= [1]*n )
    
    df = df[['is-target','is-infector','is-informer']+cols] # replace w usercols if want
    ndf = df[df['is-informer'] != 1]


    if h_i == 0:
        all_udf = pd.concat( [ndf, best_informers_df ], axis = 0 )
        all_udf.insert(0,column = 'hashtag', value= [hashtag]*len(all_udf) )
    else:   
        temp_df = pd.concat( [ndf, best_informers_df ], axis = 0 )
        temp_df.insert(0,column = 'hashtag', value= [hashtag]*len(temp_df) )
        all_udf = pd.concat( [all_udf, temp_df], axis = 0 )


    print(f'\n\nAdded {hashtag} to DATAFRAME\n----------------------------------------------------------------')



Added avengers to DATAFRAME
----------------------------------------------------------------


Added blm to DATAFRAME
----------------------------------------------------------------


Added brexit to DATAFRAME
----------------------------------------------------------------


Added climatechange to DATAFRAME
----------------------------------------------------------------


Added covid to DATAFRAME
----------------------------------------------------------------


Added gaza to DATAFRAME
----------------------------------------------------------------


Added loveisland to DATAFRAME
----------------------------------------------------------------


Added monkeypox to DATAFRAME
----------------------------------------------------------------


Added nhs to DATAFRAME
----------------------------------------------------------------


Added olivianewtonjohn to DATAFRAME
----------------------------------------------------------------


Added supercup to DATAFRAME
-----------------------

In [8]:
all_udf.to_csv('multisource_analysis/score_db/ALL_BEST_FEATURES.csv')

# TWEETS

In [ ]:

hashtags = ['avengers','borisjohnson','blm','brexit','climatechange','covid','gaza','loveisland','monkeypox','nhs','olivianewtonjohn','supercup','UkraineWar']


for h_i,hashtag in enumerate(hashtags):

    cats = ['sentiment', 'topic', 'topic_single', 'irony', 'emoji', 'offensive','gender']

    df = pd.read_csv(f'multisource_analysis/data/{hashtag}_scored_tweets.csv')
    df.drop(['Unnamed: 0', 'text','hashtag','user_id']+cats, inplace = True, axis =1)

    # readability statistics are nan when the tweet contains no text. so change these scores to 0
    reads = ['ARI','LIX','RIX','complex_words','Coleman-Liau', 'FleschReadingEase', 'GunningFogIndex', 'LIX', 'SMOGIndex', 'RIX', 'DaleChallIndex']
    for r in reads:
        df[[r]] = df[[r]].fillna(value=0).copy()

    cases = list(set(df['target'].tolist()))

    infectors = df[df['is-target']==1]
    informers = df[df['is-informer']==1]

    n = len(infectors)

    # clean a little
    for sub in [infectors,informers]:
        sub.drop(['is-target','is-infector','is-informer'], inplace = True, axis =1)

    cols = infectors.columns.tolist()

    best_formers = []
    av_formers = []

    for case_id in cases:

        o_fect = infectors.copy()[infectors['target']==case_id]
        fect = o_fect.copy().drop('target', axis=1).to_numpy()
        o_formers = informers.copy()[informers['target']==case_id]
        formers = o_formers.copy().drop('target', axis=1).to_numpy()

        av_former = informers.copy()[informers['target']==case_id]
        av_former = av_former.copy().drop('target', axis=1).mean().to_numpy()

        # get the closest feature to the target per feature
        best_calc = abs(fect - formers)
        best_idx = [ np.argmin(abs(row.astype(float))) for row in best_calc.T ]
        best_former = np.array( [form[best_idx[i]] for i,form in enumerate(formers.T) ] ) # we keeep the informer with the closer scores per feature
        
        av_formers.append(av_former.tolist())
        best_formers.append(best_former.tolist())

    cols.remove('target')

    infectors.drop('target', inplace = True, axis =1)
    best_informers_df = pd.DataFrame(best_formers, columns = cols)

    infectors.insert(0,column = 'label', value= [1]*n )
    best_informers_df.insert(0,column = 'label', value= [0]*n )
    

    if h_i == 0:
        all_df = pd.concat( [infectors, best_informers_df ], axis = 0 )
    else:   
        all_df = pd.concat( [all_df, infectors, best_informers_df ], axis = 0 )

    

    print(f'\n\nAdded {hashtag} to DATAFRAME\n----------------------------------------------------------------')